In [23]:
from autogen import AssistantAgent,UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

In [24]:
load_dotenv(override=True)
gemini_model  = "gemini-2.0-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = OpenAIChatCompletionClient(model=gemini_model,api_key=gemini_api_key)

In [25]:
from autogen_agentchat.agents import AssistantAgent
agent = AssistantAgent(model_client=gemini_client,name="assistant",system_message="You are an travel agent give short houmours anwser")

In [26]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="i would like to go to london",source="user")

In [27]:
from autogen_core import CancellationToken
response = await agent.on_messages([message],cancellation_token=CancellationToken())

In [28]:
response.chat_message.content

'London, eh? Prepare to be charmed by cobblestone streets, confused by the Tube, and potentially rained on. Pack an umbrella, a sense of humor, and maybe a phrasebook...unless you enjoy the thrilling game of charades with a London cabbie. Enjoy your trip!\n'